## Set up lightning-cli's

In [1]:
# these variables are defined in the flake.nix
path_to_lightning = ! echo $PATH_TO_LIGHTNING
lightning_bin_dir = ! echo $LIGHTNING_BIN_DIR

# if "list index out of range", make sure you set where your nodes and lightning bins are
path_to_lightning = path_to_lightning[0]
lightning_bin_dir = lightning_bin_dir[0]

l1 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l1'
l2 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l2'

print("l1's lightning-cli is: ", l1)
print("l2's lightning-cli is: ", l2)


l1's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l1
l2's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l2


## Swapping 

"most important component of the Cashu system"

We will mint one 2 sat token, and then swap it for 2 1 sat tokens.

## Mint

Pay a lightning invoice in exchange for signed tokens. For more see [mint_melt.ipynb](./mint_melt.ipynb)

In [2]:
import json
from coincurve import PrivateKey, PublicKey
from random import randint
from lib import construct_inputs, generate_blinded_messages

In [3]:
pubkeys = !$l1 cashu-get-keys | jq -r .keysets[0].keys
pubkeys = json.loads(''.join(pubkeys))

# only active for dev environments
privkeys = !$l1 cashu-dev-get-privkeys
privkeys = json.loads(''.join(privkeys))

In [4]:
mint_quote = !$l1 -k cashu-quote-mint amount=2 unit="sat" | jq -r 
mint_quote = json.loads(''.join(mint_quote))

# quote is the unique id for this exchange
mint_id = mint_quote["quote"]
# `request` is a lightning invoice
mint_invoice = mint_quote["request"]

print("quoteId:", mint_id)
print("bolt11: ", mint_invoice)

quoteId: 30f012662a43ba92
bolt11:  lnbcrt20n1pj6kztusp5jm9ulavuncplkktjewclpyjxhc8j2gtghdms6r5tjwpfr0qk8agqpp5k3wpwtpahy5a4mdcl9w9as0st0sxyqn27v4dq5d2wqafglr4ax8sdqsg9hzq6twwehkjcm9xqyjw5qcqp2fp4ppygh3980ddz3nqmcp8axes6xux788sntnr6vu7sh5ndl0avxz2yq9qx3qysgqnxl5szyqjrplrlvnhqhlpzjlqatr43jeumraz6qwzjpwmgn6kmxhw4ucmjmegvxtf0lzv9y76zvjg5qa2txjple6c9axjy0lyqh3krqpuqduah


In [5]:
# pay the invoice so l1 gives us the tokens
!$l2 pay $mint_invoice

{
   "destination": "02dd1023c37ed5d4e43b0754efcb73d6e457e06fd40fd43461e078b82fe360c1de",
   "payment_hash": "b45c172c3db929daedb8f95c5ec1f05be062026af32ad051aa703a947c75e98f",
   "created_at": 1705707900.172246288,
   "parts": 1,
   "amount_msat": 2000,
   "amount_sent_msat": 2000,
   "payment_preimage": "ae3fbfabc0ed9d8153eb8be21f99972fcaffb9cd103b4ced4e28d189adff0cc8",
   "status": "complete"
}


In [6]:
secrets = [str(randint(0, 10000)) for _ in range(1)]
# one 2 sat token
amounts = [2]
b_messages, rs = generate_blinded_messages(secrets, amounts)
b_messages

[{'amount': 2,
  'B_': '0324fab2bc5ffb6dbaa78afa06400c45a1afc0b0842403b5a25d4dbd261a8d0934'}]

### Mint token

In [7]:
# lightning-cli -k cashu-mint quote=<str> blinded_messages=<List[blinded_messages]>
mint_command = f'{l1} -k cashu-mint quote=\'{mint_id}\' blinded_messages=\'{json.dumps(b_messages)}\''
blinded_sigs = !$mint_command
blinded_sigs = json.loads(''.join(blinded_sigs))

# blinded_sigs should be a list
if isinstance(blinded_sigs, dict):
    raise AssertionError("{}".format(blinded_sigs))
else:
    print(blinded_sigs)

[{'amount': 2, 'id': '00f775c2e5e81aa3', 'C_': '023d5343e294b6b443700da6b75c3ad5a6bb7cb79bf192afc590febc9b3b5230b2'}]


### Generate new blinded messages to swap


In [10]:
swap_secrets = [str(randint(0, 10000)) for _ in range(2)]
# two 1 sat tokens
swap_amounts = [1, 1]
swap_b_messages, swap_rs = generate_blinded_messages(swap_secrets, swap_amounts)

inputs = construct_inputs(blinded_sigs,rs, secrets, pubkeys, privkeys)
inputs

'[{"amount": 2, "C": "030ecc9dc34b2b2d533c5972e59ff6ea79b32767be0c67a4a8d319331ef8467639", "id": "00f775c2e5e81aa3", "secret": "9673"}]'

In [9]:
# lightning-cli -k cashu-swap inputs=<List[BlindedMessages]> outputs=<List[Proofs]>
swap_command = f'{l1} -k cashu-swap inputs=\'{inputs}\' outputs=\'{json.dumps(swap_b_messages)}\''
!$swap_command

[
   {
      "amount": 1,
      "id": "00f775c2e5e81aa3",
      "C_": "02151119ded4b930cfefc5600b2890698450acc12388abf18bf89a6540c7719107"
   },
   {
      "amount": 1,
      "id": "00f775c2e5e81aa3",
      "C_": "03b6e5665c9128b2b1feef181bf14928820384c5037eaf5d29816f1cb376bb24a1"
   }
]
